In [4]:
import os
import redshift_utils as rs
import pandas as pd
import time
import sagemaker
import boto3
from skrub import TableReport
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
bucket = "edcs3.prod.biads"
boto_session = boto3.session.Session(region_name="eu-west-1")
sess = sagemaker.session.Session(boto_session, default_bucket=bucket)
region = sess.boto_session.region_name
account = sess.boto_session.client('sts').get_caller_identity()['Account']
role = sagemaker.get_execution_role()
ssm_client = boto3.client('ssm')
kms_key = ssm_client.get_parameter(Name='/prod/sagemaker/BIADS/sloth/KMS_KEY_ID', WithDecryption=True)["Parameter"]["Value"] 

In [6]:
def load_data(params, sql):
    sloth_credentials = rs.get_sloth_credentials()
    con = rs.connect_to_redshift(sloth_credentials)
    data = rs.execute_sql(
        RSconn=con,
        sql=sql,
        input_type="file",
        params=params
    )
    return data

In [7]:
# Create prep table
params = {'@iam': 'arn:aws:iam::765819017647:role/rs-edcs3.prod.marketing',
          '@kms': kms_key, 
         }
data = load_data(params=params, sql='../data/customer_market_group_analysis.sql')
df = pd.DataFrame(data)
